## FDW Crop Production Data Profiling - Uganda

In [1]:
import os, sys, glob, json
from itertools import product, compress, chain
from functools import reduce
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import requests
import numpy as np
import pandas as pd
import geopandas as gpd
from tools import save_hdf, save_npz, load_npz, PrintAdminUnits, PlotAdminShapes
from tools import FDW_PD_Sweeper, FDW_PD_AvalTable, FDW_PD_Compiling, FDW_PD_GrainTypeAgg, FDW_PD_ValidateFnidName
from tools import FDW_PD_CreateAdminLink, FDW_PD_RatioAdminLink, FDW_PD_ConnectAdminLink
from tools_graphic import PlotBarProduction, PlotLinePAY, PlotHeatCropSystem, PlotHeatSeasonData
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
pd.options.mode.chained_assignment = None

In [2]:
# CPCV2 grain code ------------------------------ #
grain_code = pd.read_hdf('./data/crop/grain_cpcv2_code.hdf')
product_category = grain_code[['product', 'product_category']].set_index('product').to_dict()['product_category']
# ----------------------------------------------- #

# Load FEWS NET administrative boundaries ------- #
epsg = 'epsg:32736' # Uganda
fn_shapes = sorted(glob.glob('./data/shapefile/fewsnet/UG_Admin?_????.shp'))
shape_all = []
for fn in fn_shapes:
    name = fn[-18:-4]
    exec('%s = gpd.read_file("%s").to_crs("%s")' % (name, fn, epsg))
    exec('%s["area"] = %s["geometry"].area/10**6' % (name, name))
    exec('shape_all.append(%s)' % (name))
shape_all = pd.concat(shape_all, axis=0).reset_index(drop=True)
PrintAdminUnits(shape_all)
# ----------------------------------------------- #

# FDW API host address -------------------------- #
host = 'https://fdw.fews.net'
auth = tuple(json.loads(open('token.json', "r").read()))
parameters = {
    'format': 'json',
    'country': 'Uganda',
    'product': 'R011',
    'survey_type': 'crop:best'
}
endpoint = '/api/cropproductionindicatorvalue/'
response = requests.get(host + endpoint, auth=auth, params=parameters, proxies={})
response.raise_for_status()
df = pd.DataFrame.from_records(response.json())
df_origin = df.copy()
# ----------------------------------------------- #

# Manual Pre-processing before Sweeping --------- #
# 1. Default setting 
# a) None-type population group
df.loc[df['population_group'].isna(), 'population_group'] = 'none'
df.loc[df['population_group'] == '', 'population_group'] = 'none'
# ----------------------------------------------- #

# FDW Production Data Inspection ---------------- #
df, df_raw = FDW_PD_Sweeper(df)
table_dict = FDW_PD_AvalTable(df, shape_all)
# ----------------------------------------------- #

# FEWS NET Shapefile comparison ----------------- #
shape_used = pd.concat([UG_Admin2_1990, UG_Admin2_2007], axis=0)
PlotAdminShapes(shape_used, label=True)
# ----------------------------------------------- #

- FEWS NET admin shapefiles ------------------- #
| year	 | Admin1   | # units   | Admin2   | # units   |
| 1990	 | UG1990A1 | 4	| UG1990A2	| 34	|
| 1991	 | UG1991A1 | 4	| UG1991A2	| 38	|
| 1994	 | UG1994A1 | 4	| UG1994A2	| 39	|
| 1997	 | UG1997A1 | 4	| UG1997A2	| 45	|
| 2001	 | UG2001A1 | 4	| UG2001A2	| 56	|
| 2005	 | UG2005A1 | 4	| UG2005A2	| 69	|
| 2007	 | UG2007A1 | 4	| UG2007A2	| 80	|
| 2010	 | UG2010A1 | 4	| UG2010A2	| 112	|
| 2014	 | UG2014A1 | 4	| UG2014A2	| 112	|
| 2021	 | UG2021A1 | 4	| UG2021A2	| 135	|
----------------------------------------------- #
- Remove missing records ---------------------- #
Orignial data points: 2,772
Removed 743 "Not Collected" points
0/693 "Area Harvested" points are retained.
686/693 "Area Planted" points are retained.
674/693 "Quantity Produced" points are retained.
669/693 "Yield" points are retained.
Current data points: 2,029

- Minor changes are applied.. ----------------- #

- Basic information --------------------------- #
Data period: 19

ValueError: *** Process is stopped ***

- Uganda crop seasonal calendar

![FEWS NET](https://fews.net/sites/default/files/styles/large/public/seasonal-calendar-uganda.png?itok=ka9niXoM)

- FDW data consists of `UG1990A2` and `UG2007A2`.

| Year  | Admin-1  | # units  | Admin-2  | # units |
| :---: | :----:   | :----:   | :----:   | :---:   |
| 1990  | UG1990A1 | 4        | **`UG1990A2`** | 34      |
| 1991  | UG1991A1 | 4        | UG1991A2 | 38      |
| 1994  | UG1994A1 | 4        | UG1994A2 | 39      |
| 1997  | UG1997A1 | 4        | UG1997A2 | 45      |
| 2001  | UG2001A1 | 4        | UG2001A2 | 56      |
| 2005  | UG2005A1 | 4        | UG2005A2 | 69      |
| 2007  | UG2007A1 | 4        | **`UG2007A2`** | 80      |
| 2010  | UG2010A1 | 4        | UG2010A2 | 112     |
| 2014  | UG2014A1 | 4        | UG2014A2 | 112     |
| 2021  | UG2021A1 | 4        | UG2021A2 | 135     |

- Comparison between admin boundaries.

![image](https://github.com/chc-ucsb/GlobalCropData/blob/main/figures/UG_admin_shapes.png?raw=true)

- **FDW data consists of only 5 years of records**.
- **`UG2007A2`** is used to represent the current admin-level 2 crop data.
- Uganda has three crop seasons: `Annual harvest`,`First harvest` and `Second harvest`.
- Uganda has no crop production system.
- Uganda has no population group.

In [5]:
# Define the latest shapefile ------------------- #
latest_level = 2
shape_latest = UG_Admin2_2007.copy().to_crs('epsg:4326')
# ----------------------------------------------- #

# # Validation of FNIDs and Names ----------------- #
# df = FDW_PD_ValidateFnidName(df, shape_used, shape_latest)
# # ----------------------------------------------- #

# # FDW Production Data Compiling ----------------- #
# area, prod = FDW_PD_Compiling(df, shape_used)
# area_all, prod_all = area.copy(), prod.copy()
# mdx_pss = area.columns.droplevel([0,1]).unique()
# # ----------------------------------------------- #